# Connecting to Marvel's API
The Marvel Developer Portal allows developers everywhere to access information about Marvel's vast library of comics, characters, movies...

The API uses a an API Key authentification method, so the first step to connect to it is logging into the portal and generating a key with your email address. 

In [ ]:
# Import several libraries that will be used later

import hashlib
import pandas as pd
import requests
import time

## Connecting to the Marvel API

In [ ]:
pub_api_key = "6fc395768fb1406cb953039666f92d8e"
priv_api_key = "c3d47d8c38d2ff96fc5087726be1173e35610d63"

In [ ]:
ts = str(time.time())

In [ ]:
hash_val = (ts + priv_api_key + pub_api_key).encode()
md5_hash = hashlib.md5(hash_val).hexdigest()

### 1) Provide a list of 30 Marvel characters

In [ ]:
characters_url = "http://gateway.marvel.com/v1/public/characters"

characters_params = {
    "apikey": pub_api_key,
    "ts": ts,
    "hash": md5_hash,
    "limit": 30
}

char_response = requests.get(url = characters_url, params = characters_params).json()

In [ ]:
char_data = dict(char_response)["data"]["results"]

In [ ]:
character_names = []

for hero in char_data:
    character_names.append(hero["name"])
    
character_names

### 2) Retrieve the Ids for all the characters in your list (in str form)

In [ ]:
# continuation of q1, thus also using variables from that question

character_ids = []

for hero in char_data:
    character_ids.append(str(hero["id"]))
    
character_ids

### 3) Retrieve the total number of Events available for all the characters in your list (in integer form)

In [ ]:
# continuation of q1, thus also using variables from that question

character_events_available = []

for hero in char_data:
    character_events_available.append(hero["events"]["available"])
    
character_events_available

### 4) Retrieve the total number of Series available for all the characters in your list  (in integer form)

In [ ]:
# continuation of q1, thus also using variables from that question

character_series_available = []

for hero in char_data:
    character_series_available.append(hero["series"]["available"])
    
character_series_available

### 5) Retrieve the total number of Comics available for all the characters in your list (in integer form)

In [ ]:
# continuation of q1, thus also using variables from that question

character_comics_available = []

for hero in char_data:
    character_comics_available.append(hero["comics"]["available"])
    
character_comics_available

### 6) Retrieve the Price of the most expensive comic that the character was featured in for all the characters in your list (in float form and USD)

In [ ]:
expensive_price = []

for i in range(0,30):  
    if character_comics_available[i] == 0:
        expensive_price.append(0.0)
                   
    else:
        url_price = 'http://gateway.marvel.com/v1/public/characters/' + character_ids[i] + '/comics'
        offset = 0
        num_comics = character_comics_available[i]
        
        price_info = []
        prices = []
        
        while num_comics > 0:
            params_price = {'apikey': pub_api_key, 'ts': ts, 'hash': md5_hash, 'limit': 100, 'offset': offset}
            response6 = requests.get(url = url_price, params = params_price).json()
            data6 = response6['data']['results']

            for n in data6:
                price_info.append(n['prices'])
              
                for x in price_info:
                    for y in x:
                        prices.append(y['price'])
                            
            offset = offset + 100
            num_comics = num_comics - 100
        
        expensive_price.append(max(prices))
                   

expensive_price

### 7) Store the data above in a pandas DataFrame called df containing exactly in the following columns: 
Character Name, Character ID, Total Available Events, Total Available Series, Total Available Comics, Price of the Most Expensive Comic. 

If a character is not featured in Events, Series or Comics the corresponding entry should be filled in with a None (of NoneType). If a character does not have a Price the corresponding entry should be filled in with a None (of NoneType).

In [ ]:
raw_data = {
    "Character Name": character_names,
    "Character ID": character_ids,
    "Total Available Events": character_events_available,
    "Total Available Series": character_series_available,
    "Total Available Comics": character_comics_available,
    "Price of the Most Expensive Comic": expensive_price
}

df = pd.DataFrame(raw_data)
df.head()

In [ ]:
df.replace(0, None, inplace = True)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df = df.astype({
    "Total Available Events": "Int64",
    "Total Available Series": "Int64",
    "Total Available Comics": "Int64",
    "Price of the Most Expensive Comic": "Float64"
})

# Note, since the API outputs "missing values" as 0, they can be changed to None using replace. However, this 
# changes the data type of all columns to object, contradicting with the earlier questions. Best solution appears 
# to be changing the "None" typed to pandas built in "NAType" which allows the coexistence of Int64 and NAN ^^

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.to_csv("data.csv", index = False)